In [1]:
# Python program to generate word vectors using Word2Vec
  
# importing all necessary modules
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import warnings 
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec

  
# import basic packages
import pandas as pd
import numpy as np
import os
import time
import datetime

# clean txt
import string
import re

# cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to C:\Users\Jinhang
[nltk_data]     Jiang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print (os.getcwd())
os.chdir('D:/OneDrive/ASU/2021 Spring/Applied Project/ASU_Applied_Project_2021/Text analytics')
print (os.getcwd())

C:\Users\Jinhang Jiang\ASU - Adidas
D:\OneDrive\ASU\2021 Spring\Applied Project\ASU_Applied_Project_2021\Text analytics


In [81]:
#file = pd.read_csv("TextAnalytics.csv")

In [45]:
#file1 = pd.read_csv("txt_data_full.csv")
#file2 = pd.read_csv("txt_data_korean.csv")
#file=pd.read_csv("txt_data_full.csv") 

In [82]:
#view = file.groupby(["Celebrity","txt"]).size().reset_index(name='Freq')
view = file.groupby(["Celebrity","MsgBody"]).size().reset_index(name='Freq')

In [15]:
#file1 = file1[file1["Celebrity"]!='BlackPink']
#file1 = file1[file1["Celebrity"]!='Naeun Son']
#file1.Celebrity.value_counts()

In [16]:
#len(file1)+len(file2)

In [255]:
#file = pd.concat([file1,file2],axis=0)

In [6]:
#file.to_csv("FullData.csv",index=False)

In [83]:
view.Celebrity.unique()

array(['AdrieneMishler', 'BadBunny', 'Beyonce', 'BlackPink',
       'KarlieKloss', 'KerwinFrost', 'Naeun', 'NinjaHyper',
       'PharrellWilliams', 'YaraShahidi', 'ZoeSaldana'], dtype=object)

In [84]:
#col = ['Celebrity','txt']
col = ['Celebrity','MsgBody']
df = view[col]
df.columns = ['Celebrity','txt']

In [85]:
df.isnull().sum()

Celebrity    0
txt          0
dtype: int64

In [86]:
#df.to_csv("uniquesubredditdata.csv",index=False)

In [87]:
df.Celebrity.value_counts()

BadBunny            971
BlackPink           955
Beyonce             929
KarlieKloss         859
NinjaHyper          829
PharrellWilliams    462
Naeun               428
ZoeSaldana          279
YaraShahidi         165
KerwinFrost         149
AdrieneMishler        4
Name: Celebrity, dtype: int64

In [52]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [88]:
df["Clean_txt"] = df.txt.apply(text_preproc)

In [89]:
df_merge = df.groupby(['Celebrity'])['Clean_txt'].apply(list).groupby(level=0).apply(list)
#df_merge = df.groupby(['Celebrity'])['txt'].apply(lambda x: '\n'.join(x.astype(str))).reset_index()
df_merge = pd.DataFrame(df_merge)
df_merge["clean txt"] = df_merge.Clean_txt.tolist()
df_merge

,Clean_txt,clean txt
Celebrity,,
AdrieneMishler,"[[ , lot intense expected im loving new serie...","[[ , lot intense expected im loving new serie..."
BadBunny,"[[ looks great really , remindme one week , ...","[[ looks great really , remindme one week , ..."
Beyonce,[[ link info feedback savevideo dmca removal r...,[[ link info feedback savevideo dmca removal r...
BlackPink,[[ ros actually live vote inkigayo today short...,[[ ros actually live vote inkigayo today short...
KarlieKloss,"[[ , , wow , post likely made spam bot spam...","[[ , , wow , post likely made spam bot spam..."
KerwinFrost,"[[ dont know find hard believe lol , theory a...","[[ dont know find hard believe lol , theory a..."
Naeun,"[[ , , september issue gloomy day , clip al...","[[ , , september issue gloomy day , clip al..."
NinjaHyper,[[ virginity vrjind learn pronounce state neve...,[[ virginity vrjind learn pronounce state neve...
PharrellWilliams,"[[ till morning mary j blige sooo rare , skat...","[[ till morning mary j blige sooo rare , skat..."


In [90]:
s = df_merge.Clean_txt
f = s.tolist()

In [91]:
len(f)

11

In [92]:
# Replaces escape character with space  
data = []
  
# iterate through each sentence in the file
for h in range(len(f)):
    for i in sent_tokenize(str(f[h])):
        temp = []
        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())
    data.append(temp)

In [58]:
#data = word_tokenize(str(f))

This Google Developers blog post says:

Well, the following "formula" provides a general rule of thumb about the number of embedding dimensions:

embedding_dimensions =  number_of_categories**0.25

That is, the embedding vector dimension should be the 4th root of the number of categories.

Interestingly, the Word2vec Wikipedia article says (emphasis mine):

Nevertheless, for skip-gram models trained in medium size corpora, with 50 dimensions, a window size of 15 and 10 negative samples seems to be a good parameter setting.

Assuming a standard-ish sized vocabulary of 1.5 million words, this rule of thumb comes surprisingly close:

50 == 1.5e6 ** 0.2751

Parameters:
https://radimrehurek.com/gensim/models/word2vec.html

In [93]:
num_words = len(str(df_merge['clean txt'].tolist()))
num_words

328639

In [94]:
dim_size = num_words**0.25
"{:.8f}".format(float(dim_size))

'23.94306669'

In [148]:
# Create skip-gram model
model1 = gensim.models.Word2Vec(data, 
                                min_count = 1, 
                                size = 24, 
                                window = 15, 
                                negative= 5,
                                sg=1)
model1.train(data, total_examples=model1.corpus_count,epochs=10)

(465751, 664420)

In [ ]:
vocab, vectors = model1.wv.key_to_index, model1.wv.get_normed_vectors()

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1]) for v in vocab.items()])

# init dataframe using embedding vectors and set index as node name
skip_gram =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
skip_gram.index = name_index[:,0]

In [149]:
vocab, vectors = model1.wv.vocab, model1.wv.vectors

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1].index) for v in vocab.items()]) #.index

# init dataframe using embedding vectors and set index as node name
skip_gram =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
skip_gram.index = name_index[:,0]

In [150]:
skip_gram

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
[,0.448929,0.352467,-0.277827,-1.025365,-0.286233,0.313107,-1.011702,0.397680,-0.178801,-1.191547,...,-0.483522,-0.329058,-0.723705,-0.353244,-0.298426,0.077839,0.476473,0.413763,0.124413,0.050481
',0.309252,0.019247,0.664373,0.066490,-0.237804,0.340768,-0.560672,0.770583,-0.120820,0.215736,...,-0.463441,-0.021508,-0.070747,-0.225182,-0.473421,-0.488701,0.368802,0.693047,-0.442766,0.151568
",",0.271280,-0.019748,0.711261,0.078265,-0.222964,0.419945,-0.508146,0.858155,-0.155643,0.210081,...,-0.443286,-0.098165,-0.107587,-0.239694,-0.397560,-0.431915,0.254447,0.740652,-0.434310,0.171679
lot,0.691905,0.186163,0.179255,-0.375637,-0.174451,-0.236841,-0.936031,-0.003984,-0.790878,0.466180,...,-0.093743,-0.742182,0.453720,-0.744413,0.139190,-0.106973,-0.084066,0.140198,-0.359141,-0.486872
intense,0.163283,0.209086,0.071498,-0.269873,-0.300607,0.357290,-0.737919,-0.078279,-0.432388,0.312307,...,-0.091036,-0.292140,0.149373,-0.230462,0.094319,-0.217423,0.151339,0.355185,0.020076,-0.295085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
geekboners,0.251129,0.289463,0.152746,-0.139247,-0.093508,0.538443,-0.281336,0.461496,-0.197836,-0.071337,...,-0.179215,-0.094231,-0.065529,-0.191938,-0.581866,-0.417033,0.344111,0.721855,-0.295043,0.091944
ure,0.166506,0.245509,0.424546,-0.469263,0.105074,0.605109,-0.501088,0.558202,-0.560962,-0.017111,...,-0.252148,-0.101439,-0.266395,-0.373530,-0.775347,-0.380534,0.389573,0.814459,-0.249236,0.078667
oyu,0.131441,0.281625,0.274461,-0.231422,-0.056382,0.446756,-0.367384,0.402709,-0.454290,0.105018,...,-0.198068,-0.199011,-0.092205,-0.308976,-0.503124,-0.296953,0.244564,0.690902,-0.202603,0.109299
dood,0.177851,0.281395,0.356482,-0.109070,-0.067447,0.421425,-0.244381,0.551277,-0.340765,0.238306,...,-0.342369,-0.114313,0.080162,-0.334481,-0.416072,-0.206581,0.311295,0.681672,-0.355574,0.146082


In [151]:
print(model1.wv.similarity('kpop','idol'))
print(model1.wv.similarity('kpop','movie'))
print(model1.wv.similarity('kpop','fortnite'))
print(model1.wv.similarity('ninja','fortnite'))
print(model1.wv.similarity('ninja','idol'))
print(model1.wv.similarity('ninja','kpop'))

0.70133483
0.45662332
0.21029365
0.8811493
0.6051391
0.35325238


In [154]:
model1.wv.most_similar('ninja',topn=15)

[('fortnite', 0.8811492919921875),
 ('minecraft', 0.8775534629821777),
 ('player', 0.869595468044281),
 ('facts', 0.8321744799613953),
 ('sellout', 0.8300344944000244),
 ('dumb', 0.8273787498474121),
 ('dis', 0.8273749351501465),
 ('gaming', 0.8243447542190552),
 ('blevins', 0.8234016299247742),
 ('mixer', 0.8189645409584045),
 ('stream', 0.8184990286827087),
 ('streak', 0.8113013505935669),
 ('twitch', 0.8102708458900452),
 ('himself', 0.8098055124282837),
 ('frick', 0.8041304349899292)]

In [155]:
def flatten(data):
    return " ".join([str(item) for var in data for item in var])
embedding = []
embedding = pd.DataFrame(embedding)
start = time.time()
for i in range(len(df_merge.index)):
    print('Round #',i+1,'for:',df_merge.index[i],"started")
    temp_list = []
    res = 0
    temp_df = []
    score = []
    temp_list = pd.DataFrame(temp_list)
    for j in word_tokenize(flatten(df_merge['clean txt'][i])):
        if j in skip_gram.index:
            try:
                temp_list = temp_list.append(skip_gram.loc[[j]])
            except:
                pass
    res = [temp_list.mean(axis=0)]
    res = pd.DataFrame(res)
    res.reset_index(drop=True, inplace=True)
    temp_df = pd.Series(df_merge.index[i])
    #temp_df.reset_index(drop=True, inplace=True)
    score = pd.concat([temp_df, res], axis=1)
    embedding = embedding.append(score)
    print('Round #',i+1,'for:',df_merge.index[i],"completed")
end = time.time()
sec = end - start
str(datetime.timedelta(seconds=sec))

Round # 1 for: AdrieneMishler started
Round # 1 for: AdrieneMishler completed
Round # 2 for: BadBunny started
Round # 2 for: BadBunny completed
Round # 3 for: Beyonce started
Round # 3 for: Beyonce completed
Round # 4 for: BlackPink started
Round # 4 for: BlackPink completed
Round # 5 for: KarlieKloss started
Round # 5 for: KarlieKloss completed
Round # 6 for: KerwinFrost started
Round # 6 for: KerwinFrost completed
Round # 7 for: Naeun started
Round # 7 for: Naeun completed
Round # 8 for: NinjaHyper started
Round # 8 for: NinjaHyper completed
Round # 9 for: PharrellWilliams started
Round # 9 for: PharrellWilliams completed
Round # 10 for: YaraShahidi started
Round # 10 for: YaraShahidi completed
Round # 11 for: ZoeSaldana started
Round # 11 for: ZoeSaldana completed


'0:00:58.202160'

In [157]:
embedding.columns=list(range(skip_gram.shape[1]+1))
embedding.rename(columns={ embedding.columns[0]: "Celebrity" }, inplace = True)
embedding

,Celebrity,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,AdrieneMishler,0.310744,0.114894,0.372717,-0.208280,-0.290493,0.277001,-0.859835,0.364915,-0.406649,...,-0.437173,-0.160311,0.236366,-0.228445,-0.033324,-0.019452,0.036492,0.394891,-0.101621,-0.199122
0,BadBunny,0.136958,0.674445,0.281728,0.049650,-0.211023,0.449171,-0.172975,0.523590,-0.355485,...,-0.413494,-0.498217,0.240107,-0.301446,0.023046,-0.028627,0.007682,0.711466,-0.004052,0.522069
0,Beyonce,0.059381,0.264879,0.358117,-0.372402,-0.354545,0.326644,-0.584422,0.580935,-0.405819,...,-0.288656,-0.077561,0.311651,-0.279049,-0.044278,-0.201773,-0.047545,0.493065,-0.204366,-0.127300
0,BlackPink,0.191954,0.089561,0.261991,-0.328939,-0.338810,0.142819,-0.558399,0.471002,-0.617489,...,-0.274308,-0.249293,0.160287,-0.209325,0.032408,-0.278825,0.111223,0.603437,-0.132365,-0.265872
0,KarlieKloss,0.191123,0.230257,0.390204,-0.198018,-0.404740,0.339056,-0.387688,0.577787,-0.465359,...,-0.379826,-0.109078,0.195172,-0.283731,-0.316237,-0.254614,0.156836,0.685766,-0.450535,-0.043631
0,KerwinFrost,0.094065,0.187953,0.252109,-0.219476,-0.317549,0.410188,-0.554135,0.342787,-0.401288,...,-0.089889,-0.216912,0.256140,-0.253167,-0.065108,-0.220009,0.015704,0.499282,-0.065472,-0.030220
0,Naeun,0.152211,0.147193,0.478240,-0.231692,-0.471472,0.229709,-0.536375,0.543151,-0.478449,...,-0.256787,-0.317885,0.077743,-0.207339,-0.320811,-0.445144,0.151367,0.598740,-0.104608,-0.037189
0,NinjaHyper,0.002279,0.139899,0.239689,-0.268080,-0.182330,0.414028,-0.593595,0.468796,-0.410481,...,-0.185115,-0.176928,0.245693,-0.226624,0.066450,-0.429991,0.090329,0.397031,-0.150313,-0.239926
0,PharrellWilliams,0.200032,0.197603,0.444601,-0.330970,-0.326325,0.247158,-0.595155,0.469113,-0.421571,...,-0.269860,-0.163783,0.254522,-0.366641,-0.017411,-0.132049,-0.114197,0.442279,-0.228937,-0.039803
0,YaraShahidi,0.181733,0.070647,0.224475,-0.188440,-0.194939,0.471518,-0.444826,0.665451,-0.304680,...,-0.418837,-0.138920,0.409053,-0.170414,-0.099778,-0.270859,-0.022407,0.471398,-0.149069,-0.255373


In [284]:
#embedding.to_csv("word2vec_embedding_V6_20.csv",index=False)

In [158]:
def similarity(data,col,target,order = 'TopN'):
    #default of parameter 'order' is equal to TopN, alternative is BotN
    sim = []
    for i in data[col]:
        if i != target:
            a = data[data[col]==target].drop(col,axis=1)
            b = data[data[col]==i].drop(col,axis=1)
            temp = (i,cosine_similarity(a,b))
            sim.append(temp)
    if order == 'TopN':
        sim = sorted(sim, key=lambda t: t[1], reverse=True)
    elif order == 'BotN':
        sim = sorted(sim, key=lambda t: t[1], reverse=False)
    return sim

In [159]:
similarity(embedding,'Celebrity','Naeun','TopN')

[('ZoeSaldana', array([[0.9510789]], dtype=float32)),
 ('KarlieKloss', array([[0.9388172]], dtype=float32)),
 ('BlackPink', array([[0.9197567]], dtype=float32)),
 ('KerwinFrost', array([[0.9133661]], dtype=float32)),
 ('Beyonce', array([[0.9089148]], dtype=float32)),
 ('PharrellWilliams', array([[0.8914171]], dtype=float32)),
 ('YaraShahidi', array([[0.8830012]], dtype=float32)),
 ('AdrieneMishler', array([[0.87815624]], dtype=float32)),
 ('NinjaHyper', array([[0.866635]], dtype=float32)),
 ('BadBunny', array([[0.721035]], dtype=float32))]

In [160]:
similarity(embedding,'Celebrity','Beyonce','TopN')

[('PharrellWilliams', array([[0.97792125]], dtype=float32)),
 ('KerwinFrost', array([[0.95540553]], dtype=float32)),
 ('BlackPink', array([[0.94733673]], dtype=float32)),
 ('NinjaHyper', array([[0.93995535]], dtype=float32)),
 ('YaraShahidi', array([[0.93912035]], dtype=float32)),
 ('ZoeSaldana', array([[0.9364201]], dtype=float32)),
 ('KarlieKloss', array([[0.92646164]], dtype=float32)),
 ('AdrieneMishler', array([[0.9258249]], dtype=float32)),
 ('Naeun', array([[0.9089148]], dtype=float32)),
 ('BadBunny', array([[0.7626959]], dtype=float32))]

In [161]:
similarity(embedding,'Celebrity','NinjaHyper','TopN')

[('KerwinFrost', array([[0.94671685]], dtype=float32)),
 ('BlackPink', array([[0.94343036]], dtype=float32)),
 ('Beyonce', array([[0.93995535]], dtype=float32)),
 ('PharrellWilliams', array([[0.92128867]], dtype=float32)),
 ('YaraShahidi', array([[0.9194962]], dtype=float32)),
 ('ZoeSaldana', array([[0.9036615]], dtype=float32)),
 ('AdrieneMishler', array([[0.89098626]], dtype=float32)),
 ('Naeun', array([[0.866635]], dtype=float32)),
 ('KarlieKloss', array([[0.8648081]], dtype=float32)),
 ('BadBunny', array([[0.7125666]], dtype=float32))]

##### Pre-trained Model from Google News

In [309]:
print (os.getcwd())
os.chdir('D:/OneDrive/ASU/2021 Spring/Applied Project/ASU_Applied_Project_2021')
print (os.getcwd())

D:\OneDrive\ASU\2021 Spring\Applied Project\ASU_Applied_Project_2021\Model
D:\OneDrive\ASU\2021 Spring\Applied Project\ASU_Applied_Project_2021


In [317]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('./Model/GoogleNews-vectors-negative300.bin',binary=True)

In [318]:
vocab, vectors = model.wv.vocab, model.wv.vectors

# get node name and embedding vector index.
name_index = np.array([(v[0], v[1].index) for v in vocab.items()]) #.index

# init dataframe using embedding vectors and set index as node name
pretrain =  pd.DataFrame(vectors[name_index[:,1].astype(int)])
pretrain.index = name_index[:,0]

In [319]:
pretrain

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
</s>,0.001129,-0.000896,0.000319,0.001534,0.001106,-0.001404,-0.000031,-0.000420,-0.000576,0.001076,...,-0.001549,0.001373,-0.000061,-0.000824,0.001328,0.001160,0.000568,-0.001564,-0.000123,-0.000086
in,0.070312,0.086914,0.087891,0.062500,0.069336,-0.108887,-0.081543,-0.154297,0.020752,0.131836,...,-0.168945,-0.088867,-0.080566,0.064941,0.061279,-0.047363,-0.058838,-0.047607,0.014465,-0.062500
for,-0.011780,-0.047363,0.044678,0.063477,-0.018188,-0.063965,-0.001312,-0.072266,0.064453,0.086426,...,-0.022583,0.003723,-0.082520,0.081543,0.007935,0.000477,0.018433,0.071289,-0.034912,0.024170
that,-0.015747,-0.028320,0.083496,0.050293,-0.110352,0.031738,-0.014221,-0.089844,0.117676,0.118164,...,-0.011292,-0.015625,-0.033447,-0.020630,-0.019409,0.063965,0.020142,0.006866,0.061035,-0.148438
is,0.007050,-0.073242,0.171875,0.022583,-0.132812,0.198242,0.112793,-0.107910,0.071777,0.020874,...,-0.233398,-0.036377,-0.093750,0.182617,0.027100,0.127930,-0.024780,0.011230,0.164062,0.106934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RAFFAELE,0.009277,-0.050537,-0.018799,0.029785,0.037354,0.008911,-0.057617,0.035645,-0.052490,0.078125,...,-0.028076,-0.002853,-0.059326,-0.007385,-0.008606,-0.023560,0.016968,-0.012634,0.010620,0.026978
Bim_Skala_Bim,0.012573,0.045410,-0.043213,-0.001495,0.026489,0.049561,-0.059326,-0.083496,-0.033691,0.099609,...,0.086914,-0.041504,-0.036865,-0.023315,0.012451,-0.112305,-0.010620,-0.051514,-0.014282,0.051514
Mezze_Cafe,-0.019653,-0.090820,-0.019409,0.019653,-0.000230,0.040283,-0.050293,-0.012695,0.024170,0.013123,...,-0.046387,0.001076,-0.136719,-0.004547,0.022827,0.042969,-0.046875,-0.016357,-0.013428,0.046631
pulverizes_boulders,0.032715,-0.032227,0.036133,0.001175,-0.036621,-0.084961,0.036865,0.032471,0.063477,0.114746,...,0.020020,-0.070312,-0.167969,0.003082,-0.060303,-0.007294,-0.009827,-0.008850,0.026978,0.019043


In [323]:
model.wv.most_similar('Adidas',topn=15)

[('adidas', 0.8445298671722412),
 ('Nike', 0.7950947284698486),
 ('Adidas_ADDDY.PK_news', 0.6852840185165405),
 ('Reebok', 0.6841334104537964),
 ('Puma', 0.6774643659591675),
 ('Adidas_Salomon', 0.659113883972168),
 ('spokesman_Jan_Runau', 0.6242237091064453),
 ('adidas_Salomon_AG', 0.6223492622375488),
 ('sportswear', 0.614206075668335),
 ('Adidas_Salomon_AG', 0.6134569048881531),
 ('Umbro', 0.6109203696250916),
 ('Nike_Adidas', 0.6075664758682251),
 ('Adidas_Nike', 0.6061606407165527),
 ('Adidas_Reebok', 0.6059699058532715),
 ('Adidas_AG', 0.6053416132926941)]

In [320]:
def flatten(data):
    return " ".join([str(item) for var in data for item in var])
embedding1 = []
embedding1 = pd.DataFrame(embedding1)
start = time.time()
for i in range(len(df_merge.index)):
    print('Round #',i+1,'for:',df_merge.index[i],"started")
    temp_list = []
    res = 0
    temp_df = []
    score = []
    temp_list = pd.DataFrame(temp_list)
    for j in word_tokenize(flatten(df_merge['clean txt'][i])):
        if j in pretrain.index:
            try:
                temp_list = temp_list.append(pretrain.loc[[j]])
            except:
                pass
    res = [temp_list.mean(axis=0)]
    res = pd.DataFrame(res)
    res.reset_index(drop=True, inplace=True)
    temp_df = pd.Series(df_merge.index[i])
    #temp_df.reset_index(drop=True, inplace=True)
    score = pd.concat([temp_df, res], axis=1)
    embedding1 = embedding1.append(score)
    print('Round #',i+1,'for:',df_merge.index[i],"completed")
end = time.time()
sec = end - start
str(datetime.timedelta(seconds=sec))

Round # 1 for: Adriene Mishler started
Round # 1 for: Adriene Mishler completed
Round # 2 for: Ally Love started
Round # 2 for: Ally Love completed
Round # 3 for: BTS started
Round # 3 for: BTS completed
Round # 4 for: Bad Bunny started
Round # 4 for: Bad Bunny completed
Round # 5 for: Beyonce started
Round # 5 for: Beyonce completed
Round # 6 for: BlackPink started
Round # 6 for: BlackPink completed
Round # 7 for: Chinae Alexander started
Round # 7 for: Chinae Alexander completed
Round # 8 for: GFriend started
Round # 8 for: GFriend completed
Round # 9 for: Jerry Lorenzo started
Round # 9 for: Jerry Lorenzo completed
Round # 10 for: Karlie Kloss started
Round # 10 for: Karlie Kloss completed
Round # 11 for: Kerwin Frost started
Round # 11 for: Kerwin Frost completed
Round # 12 for: NCT started
Round # 12 for: NCT completed
Round # 13 for: Naeun Son started
Round # 13 for: Naeun Son completed
Round # 14 for: Ninjas Hyper started
Round # 14 for: Ninjas Hyper completed
Round # 15 for: Ph

'3:47:53.287497'

In [324]:
embedding1.columns=list(range(pretrain.shape[1]+1))
embedding1.rename(columns={embedding1.columns[0]: "Celebrity" }, inplace = True)
embedding1

,Celebrity,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,Adriene Mishler,0.019793,0.020356,-0.009257,0.122618,-0.088852,0.024864,0.005068,-0.067733,0.042789,...,-0.031978,0.066720,-0.075106,0.026110,-0.055297,-0.081830,-0.000275,-0.064345,0.040174,-0.035986
0,Ally Love,0.037586,0.032016,0.010924,0.101464,-0.050598,0.004588,0.039665,-0.081055,0.066994,...,-0.048690,0.056643,-0.097944,0.012078,-0.035606,-0.038087,0.015601,-0.056402,0.006107,-0.002797
0,BTS,0.033026,0.041555,0.010861,0.098711,-0.052766,0.005887,0.037012,-0.084883,0.068388,...,-0.033710,0.048913,-0.088663,0.004390,-0.036776,-0.054573,0.005194,-0.063648,0.002218,0.008100
0,Bad Bunny,0.027914,0.025996,0.023155,0.108129,-0.060856,0.019012,0.036342,-0.068683,0.077053,...,-0.029898,0.065523,-0.090585,0.017268,-0.040830,-0.042112,0.008867,-0.064537,-0.003894,0.006629
0,Beyonce,0.032351,0.023312,0.009082,0.096273,-0.056763,0.008562,0.030702,-0.083845,0.070861,...,-0.024841,0.064021,-0.099711,0.008088,-0.038623,-0.047189,0.017023,-0.067449,0.001485,0.004472
0,BlackPink,0.027150,0.032800,0.011482,0.093344,-0.065385,0.013252,0.040367,-0.078235,0.071041,...,-0.027220,0.060017,-0.080311,0.005647,-0.024289,-0.049553,0.018443,-0.076731,-0.000560,-0.002266
0,Chinae Alexander,0.008082,0.049290,0.020450,0.097467,-0.063927,0.000798,0.034737,-0.084250,0.073489,...,-0.063513,0.041224,-0.080961,0.031402,-0.030553,-0.024027,-0.011215,-0.055705,0.022407,0.003045
0,GFriend,0.027004,0.040091,0.016053,0.096585,-0.060128,0.009476,0.040476,-0.078769,0.075016,...,-0.033719,0.056597,-0.086717,0.006659,-0.035836,-0.051766,0.005225,-0.061709,-0.001846,0.009795
0,Jerry Lorenzo,0.021244,0.042060,0.008501,0.101443,-0.052247,0.001172,0.026838,-0.084881,0.071702,...,-0.029382,0.057926,-0.079298,0.019005,-0.018837,-0.042945,0.013146,-0.077336,-0.000441,0.001909
0,Karlie Kloss,0.028763,0.027666,0.018322,0.086401,-0.064500,0.008643,0.043514,-0.079579,0.077085,...,-0.029916,0.049216,-0.085180,0.015062,-0.023550,-0.044988,0.031553,-0.085584,-0.011459,-0.004492


In [325]:
#embedding1.to_csv("./Data/word2vec_embedding_pretrain1.csv",index=False)

In [343]:
similarity(embedding1,'Celebrity','Naeun Son','TopN')[0:2]

[('iZone', array([[0.9967277]], dtype=float32)),
 ('NCT', array([[0.9953402]], dtype=float32))]

In [344]:
similarity(embedding1,'Celebrity','Beyonce','TopN')[0:2]

[('Zoe Saldana', array([[0.9997813]], dtype=float32)),
 ('BlackPink', array([[0.98949885]], dtype=float32))]

In [345]:
similarity(embedding1,'Celebrity','Ninjas Hyper','TopN')[0:2]

[('Ally Love', array([[0.98195165]], dtype=float32)),
 ('NCT', array([[0.9788385]], dtype=float32))]